In [75]:
!pip uninstall openai -y
!pip install git+https://github.com/OpenGPTs-platform/openai-python

Found existing installation: openai 1.26.0
Uninstalling openai-1.26.0:
  Successfully uninstalled openai-1.26.0

  Running command git clone -q https://github.com/OpenGPTs-platform/openai-python 'C:\Users\1seba\AppData\Local\Temp\pip-req-build-imwep5qe'
You should consider upgrading via the 'C:\Projects\OpenGPTs\playground\.venv\Scripts\python.exe -m pip install --upgrade pip' command.



  Cloning https://github.com/OpenGPTs-platform/openai-python to c:\users\1seba\appdata\local\temp\pip-req-build-imwep5qe
  Resolved https://github.com/OpenGPTs-platform/openai-python to commit d304a594c582f411dedeffc8e503b74c132d5243
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for openai: filename=openai-1.26.0-py3-none-any.whl size=320295 sha256=6b584054abc34e24d659b6429227cc3c3a5a4c37f41f9db05b744aa3241324ec
  Stored in directory: C:\Users\1seba\AppData\Local\Temp\pip-ephem-wheel-cache-kjn5jyx9\wheels\60\bc\13\1fa18a5d076f25b47166014a8948258b5411234779e7a6c185
Successfully built openai


## Configure Client

In [76]:
from openai import OpenAI
import json

client = OpenAI(
    base_url="http://localhost:8000",
    api_key="api_key",
)

In [77]:
client.ops.web_retrieval.delete()

DeleteResponse(message="Collection 'web_retrieval' deleted successfully.")

## Base Demo Without Tools

In [78]:
assistant = client.beta.assistants.create(
    instructions="""Your job is to assist the assume the role of a news provider and inform the user of current news.
Always direct them to where they can learn more by providing the corresponding link.""",  # noqa
    name="News Provider Assistant",
    model="gpt-3.5-turbo",
)

In [79]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Show me interesting aerospace news from ycombinator.",  # noqa
        },
    ],
)
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order='desc'
)
print(json.dumps(messages.model_dump(), indent=2))

{
  "data": [
    {
      "id": "0798431a-7be9-4d42-a098-dbf87852f78b",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "For the latest interesting aerospace news from Y Combinator, please visit their website at https://news.ycombinator.com/"
          },
          "type": "text"
        }
      ],
      "created_at": 1718480075059,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": null,
      "status": "completed",
      "thread_id": "6b02e927-99ae-4625-8204-21a267e4924b"
    },
    {
      "id": "05d19fbe-e112-47f3-b509-f34236db43d3",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "I need to summarize the conv

## Demo With `web_retireval` Tool

In [80]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tools=[
        {"type": "web_retrieval"}
    ],
)

In [81]:
crawl = client.ops.web_retrieval.crawl_and_upsert(
    root_urls=["https://news.ycombinator.com/"],
    max_depth=1,
    description="Live news from ycombinator, a news feed centered on science and technology.",
    constrain_to_root_domain=False,
)
successful_crawls = [ci for ci in crawl.crawl_infos if ci.error is None]
print("Successful crawls: ", len(successful_crawls))
print(crawl.model_dump_json(indent=2))


Successful crawls:  45
{
  "message": "Crawling completed successfully.",
  "crawl_infos": [
    {
      "url": "https://news.ycombinator.com/",
      "error": null,
      "content": "<REMOVED>",
      "depth": 0
    },
    {
      "url": "https://news.ycombinator.com",
      "error": "Server error '503 Service Temporarily Unavailable' for url 'https://news.ycombinator.com'\nFor more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/503",
      "content": "<REMOVED>",
      "depth": 1
    },
    {
      "url": "https://news.ycombinator.com/news",
      "error": "Server error '503 Service Temporarily Unavailable' for url 'https://news.ycombinator.com/news'\nFor more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/503",
      "content": "<REMOVED>",
      "depth": 1
    },
    {
      "url": "https://news.ycombinator.com/newest",
      "error": "Server error '503 Service Temporarily Unavailable' for url 'https://news.ycombinator.com/n

In [84]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Show me interesting aerospace news from ycombinator.",  # noqa
        },
    ],
)
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order='desc'
)
print(json.dumps(messages.model_dump(), indent=2))

{
  "data": [
    {
      "id": "02d6724a-3727-471f-b4e3-6ab7e9bbc6bf",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Recent aerospace news includes Voyager 1 being fully operational and updates on AI for Math resources. You can learn more about it [here](https://terrytao.wordpress.com/2024/04/19/two-announcements-ai-for-math-resources-and-erdosproblems-com/) and [here](https://www.space.com/voyager-1-fully-operational)."
          },
          "type": "text"
        }
      ],
      "created_at": 1718480215041,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": null,
      "status": "completed",
      "thread_id": "f849bac1-7793-4588-b35b-e98d5950344e"
    },
    {
      "id": "041b37fd-3c4e-46f6-93fc-9d908997fe7d",
      "assistant_id": nul

## Demo With Tools `web_retireval` and `file_search`

In [85]:
file = client.files.create(
    purpose="assistants",
    file=open("./my_information.txt", "rb")
)
vector_store = client.beta.vector_stores.create(
    name="Information About Me",
    file_ids=[file.id],
)
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    instructions="""Your job is to assist the assume the role of a news provider and inform the user of current news.
Always direct them to where they can learn more by providing the corresponding link.
You must begin by searching through their files to find information about them.
Then you can look for relevant news.""",
    tools=[
        {"type": "web_retrieval"},
        {"type": "file_search"}
    ],
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
)
    

In [94]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Find me interesting news.",  # noqa
        },
    ],
)
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order='desc'
)
print(json.dumps(messages.model_dump(), indent=2))

{
  "data": [
    {
      "id": "0c1fce1f-164a-4694-9499-1719672f93da",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "For the latest developments in conversational AI tools leveraging transformer models and how they are impacting the field of AI, you can read more at this link: [Latest Developments in AI and Math](https://terrytao.wordpress.com/2024/04/19/two-announcements-ai-for-math-resources-and-erdosproblems-com/)"
          },
          "type": "text"
        }
      ],
      "created_at": 1718480852546,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": null,
      "status": "completed",
      "thread_id": "23146886-c999-43d5-8de6-4eabb668d0aa"
    },
    {
      "id": "05f78a69-426c-41d5-9ecb-86335ab2d212",
      "assistant_id": null,
